In [24]:
# Code largely inspired by https://www.kdnuggets.com/2020/07/pytorch-lstm-text-generation-tutorial.html

import torch
import pandas as pd
import numpy as np
import random
from collections import Counter
from nltk.tokenize import wordpunct_tokenize

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

if torch.backends.cudnn.enabled:
    torch.backends.cudnn.benchmark = False
    torch.cuda.manual_seed_all(SEED)
    
def remove_multiple_strings(cur_string, replace_list):
    for cur_word in replace_list:
        cur_string = cur_string.replace(cur_word, '')
    return cur_string

rempunc = '(),.:[]'
def clean(sentence):
    without_some_punc = remove_multiple_strings(sentence, rempunc)
    sentence = without_some_punc.lower()
    '''sentence  = sentence.replace('\n', ' nnnnnn ')
    sentence = wordpunct_tokenize(sentence)
    postagged = nltk.pos_tag(sentence)
    replace_newline = []
    for word in postagged:
        w,t = word
        if w == 'nnnnnn':
            t = 'NEWLINE'
        replace_word = w + '-' + t
        replace_newline.append(replace_word)
    return replace_newline'''
    return sentence

class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        max_epochs, batch_size, sequence_length, genre, sentiment
    ):
        self.max_epochs = max_epochs
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.genre = genre
        self.sentiment = sentiment
        self.words = self.load_words()
        self.uniq_words = self.get_uniq_words()

        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.words_indexes = [self.word_to_index[w] for w in self.words]

    def load_words(self):
        train_df = pd.read_csv('data/lyrics_sentiments.csv')
        
        lyrics = list()
        
        i=0
        while i < len(train_df.index):
            if train_df['genre'][i] == self.genre and train_df['sentiment'][i] == self.sentiment and type(train_df['lyrics'][i]) == str:
                lyrics.append(clean(train_df['lyrics'][i]))
            i += 1
         
        print(len(lyrics))
        #return ' '.join(string for string in lyrics[:100]).split(' ')
        return ' '.join(string for string in lyrics[:5000]).split(' ')

    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.words_indexes) - self.sequence_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.sequence_length]),
            torch.tensor(self.words_indexes[index+1:index+self.sequence_length+1]),
        )     

In [25]:
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader
from architecture import lstm

def train(dataset, model, max_epochs, batch_size, sequence_length):
    model.train().cuda()

    dataloader = DataLoader(dataset, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    print(len(dataloader))

    for epoch in range(max_epochs):
        state_h, state_c = model.init_state(sequence_length)

        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x.cuda(), (state_h.cuda(), state_c.cuda()))
            loss = criterion(y_pred.transpose(1, 2), y.cuda())

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })
            


In [26]:
max_epochs=10
batch_size=256
sequence_length=2
genre="Pop"
sentiment='Positive'

dataset = Dataset(max_epochs, batch_size, sequence_length, genre, sentiment)
model = lstm.Model(dataset)

16192


In [27]:
train(dataset, model, max_epochs, batch_size, sequence_length)

3774
{'epoch': 0, 'batch': 0, 'loss': 11.616735458374023}
{'epoch': 0, 'batch': 1, 'loss': 11.613432884216309}
{'epoch': 0, 'batch': 2, 'loss': 11.593783378601074}
{'epoch': 0, 'batch': 3, 'loss': 11.588079452514648}
{'epoch': 0, 'batch': 4, 'loss': 11.579544067382812}
{'epoch': 0, 'batch': 5, 'loss': 11.583211898803711}
{'epoch': 0, 'batch': 6, 'loss': 11.588349342346191}
{'epoch': 0, 'batch': 7, 'loss': 11.558064460754395}
{'epoch': 0, 'batch': 8, 'loss': 11.53449535369873}
{'epoch': 0, 'batch': 9, 'loss': 11.47711181640625}
{'epoch': 0, 'batch': 10, 'loss': 11.382307052612305}
{'epoch': 0, 'batch': 11, 'loss': 11.422560691833496}
{'epoch': 0, 'batch': 12, 'loss': 11.148287773132324}
{'epoch': 0, 'batch': 13, 'loss': 11.025195121765137}
{'epoch': 0, 'batch': 14, 'loss': 10.716794967651367}
{'epoch': 0, 'batch': 15, 'loss': 10.645244598388672}
{'epoch': 0, 'batch': 16, 'loss': 11.104520797729492}
{'epoch': 0, 'batch': 17, 'loss': 10.614532470703125}
{'epoch': 0, 'batch': 18, 'loss': 1

{'epoch': 0, 'batch': 156, 'loss': 8.731302261352539}
{'epoch': 0, 'batch': 157, 'loss': 7.999475479125977}
{'epoch': 0, 'batch': 158, 'loss': 8.018243789672852}
{'epoch': 0, 'batch': 159, 'loss': 8.300727844238281}
{'epoch': 0, 'batch': 160, 'loss': 8.512747764587402}
{'epoch': 0, 'batch': 161, 'loss': 7.467521667480469}
{'epoch': 0, 'batch': 162, 'loss': 8.053237915039062}
{'epoch': 0, 'batch': 163, 'loss': 8.239377975463867}
{'epoch': 0, 'batch': 164, 'loss': 9.07723331451416}
{'epoch': 0, 'batch': 165, 'loss': 8.321393013000488}
{'epoch': 0, 'batch': 166, 'loss': 8.445294380187988}
{'epoch': 0, 'batch': 167, 'loss': 7.8002142906188965}
{'epoch': 0, 'batch': 168, 'loss': 7.893520355224609}
{'epoch': 0, 'batch': 169, 'loss': 7.329616069793701}
{'epoch': 0, 'batch': 170, 'loss': 8.245375633239746}
{'epoch': 0, 'batch': 171, 'loss': 7.620623588562012}
{'epoch': 0, 'batch': 172, 'loss': 8.056083679199219}
{'epoch': 0, 'batch': 173, 'loss': 8.122392654418945}
{'epoch': 0, 'batch': 174, '

{'epoch': 0, 'batch': 308, 'loss': 7.686069488525391}
{'epoch': 0, 'batch': 309, 'loss': 7.488681316375732}
{'epoch': 0, 'batch': 310, 'loss': 7.1529693603515625}
{'epoch': 0, 'batch': 311, 'loss': 7.465029716491699}
{'epoch': 0, 'batch': 312, 'loss': 7.3404693603515625}
{'epoch': 0, 'batch': 313, 'loss': 7.592648983001709}
{'epoch': 0, 'batch': 314, 'loss': 7.047884464263916}
{'epoch': 0, 'batch': 315, 'loss': 6.8052077293396}
{'epoch': 0, 'batch': 316, 'loss': 6.937641620635986}
{'epoch': 0, 'batch': 317, 'loss': 8.288851737976074}
{'epoch': 0, 'batch': 318, 'loss': 7.501698970794678}
{'epoch': 0, 'batch': 319, 'loss': 7.468894004821777}
{'epoch': 0, 'batch': 320, 'loss': 7.521732330322266}
{'epoch': 0, 'batch': 321, 'loss': 7.791903972625732}
{'epoch': 0, 'batch': 322, 'loss': 7.319432258605957}
{'epoch': 0, 'batch': 323, 'loss': 7.382997512817383}
{'epoch': 0, 'batch': 324, 'loss': 7.6074395179748535}
{'epoch': 0, 'batch': 325, 'loss': 8.383562088012695}
{'epoch': 0, 'batch': 326, 

{'epoch': 0, 'batch': 460, 'loss': 7.336180686950684}
{'epoch': 0, 'batch': 461, 'loss': 7.785130977630615}
{'epoch': 0, 'batch': 462, 'loss': 8.212084770202637}
{'epoch': 0, 'batch': 463, 'loss': 6.432024955749512}
{'epoch': 0, 'batch': 464, 'loss': 10.450422286987305}
{'epoch': 0, 'batch': 465, 'loss': 8.445252418518066}
{'epoch': 0, 'batch': 466, 'loss': 7.7923903465271}
{'epoch': 0, 'batch': 467, 'loss': 9.006341934204102}
{'epoch': 0, 'batch': 468, 'loss': 8.589201927185059}
{'epoch': 0, 'batch': 469, 'loss': 8.576547622680664}
{'epoch': 0, 'batch': 470, 'loss': 7.355023384094238}
{'epoch': 0, 'batch': 471, 'loss': 7.396852016448975}
{'epoch': 0, 'batch': 472, 'loss': 7.969126224517822}
{'epoch': 0, 'batch': 473, 'loss': 8.72292709350586}
{'epoch': 0, 'batch': 474, 'loss': 7.290877819061279}
{'epoch': 0, 'batch': 475, 'loss': 7.321722030639648}
{'epoch': 0, 'batch': 476, 'loss': 7.55642032623291}
{'epoch': 0, 'batch': 477, 'loss': 7.691765785217285}
{'epoch': 0, 'batch': 478, 'los

{'epoch': 0, 'batch': 612, 'loss': 8.562941551208496}
{'epoch': 0, 'batch': 613, 'loss': 8.038656234741211}
{'epoch': 0, 'batch': 614, 'loss': 8.228861808776855}
{'epoch': 0, 'batch': 615, 'loss': 7.340128421783447}
{'epoch': 0, 'batch': 616, 'loss': 7.018761157989502}
{'epoch': 0, 'batch': 617, 'loss': 7.746171951293945}
{'epoch': 0, 'batch': 618, 'loss': 7.870079517364502}
{'epoch': 0, 'batch': 619, 'loss': 8.172679901123047}
{'epoch': 0, 'batch': 620, 'loss': 7.202449798583984}
{'epoch': 0, 'batch': 621, 'loss': 6.987987518310547}
{'epoch': 0, 'batch': 622, 'loss': 7.33712911605835}
{'epoch': 0, 'batch': 623, 'loss': 7.4231343269348145}
{'epoch': 0, 'batch': 624, 'loss': 6.904286861419678}
{'epoch': 0, 'batch': 625, 'loss': 6.724356651306152}
{'epoch': 0, 'batch': 626, 'loss': 8.21926212310791}
{'epoch': 0, 'batch': 627, 'loss': 6.960730075836182}
{'epoch': 0, 'batch': 628, 'loss': 6.873683929443359}
{'epoch': 0, 'batch': 629, 'loss': 6.575850009918213}
{'epoch': 0, 'batch': 630, 'l

...

{'epoch': 9, 'batch': 3101, 'loss': 5.057137966156006}
{'epoch': 9, 'batch': 3102, 'loss': 4.641947269439697}
{'epoch': 9, 'batch': 3103, 'loss': 4.612873554229736}
{'epoch': 9, 'batch': 3104, 'loss': 4.537210941314697}
{'epoch': 9, 'batch': 3105, 'loss': 5.105105876922607}
{'epoch': 9, 'batch': 3106, 'loss': 4.973956108093262}
{'epoch': 9, 'batch': 3107, 'loss': 5.234067916870117}
{'epoch': 9, 'batch': 3108, 'loss': 5.550563812255859}
{'epoch': 9, 'batch': 3109, 'loss': 4.212307929992676}
{'epoch': 9, 'batch': 3110, 'loss': 4.765971660614014}
{'epoch': 9, 'batch': 3111, 'loss': 5.52733850479126}
{'epoch': 9, 'batch': 3112, 'loss': 5.0860981941223145}
{'epoch': 9, 'batch': 3113, 'loss': 4.781856060028076}
{'epoch': 9, 'batch': 3114, 'loss': 5.071240425109863}
{'epoch': 9, 'batch': 3115, 'loss': 5.19161319732666}
{'epoch': 9, 'batch': 3116, 'loss': 5.23054313659668}
{'epoch': 9, 'batch': 3117, 'loss': 5.51451301574707}
{'epoch': 9, 'batch': 3118, 'loss': 4.964364051818848}
{'epoch': 9, 

{'epoch': 9, 'batch': 3254, 'loss': 4.466183185577393}
{'epoch': 9, 'batch': 3255, 'loss': 4.497129440307617}
{'epoch': 9, 'batch': 3256, 'loss': 4.982985496520996}
{'epoch': 9, 'batch': 3257, 'loss': 4.362363338470459}
{'epoch': 9, 'batch': 3258, 'loss': 5.064716815948486}
{'epoch': 9, 'batch': 3259, 'loss': 4.656639575958252}
{'epoch': 9, 'batch': 3260, 'loss': 5.021005630493164}
{'epoch': 9, 'batch': 3261, 'loss': 4.396699905395508}
{'epoch': 9, 'batch': 3262, 'loss': 4.062851905822754}
{'epoch': 9, 'batch': 3263, 'loss': 4.467896461486816}
{'epoch': 9, 'batch': 3264, 'loss': 4.3031697273254395}
{'epoch': 9, 'batch': 3265, 'loss': 4.709795951843262}
{'epoch': 9, 'batch': 3266, 'loss': 4.6064863204956055}
{'epoch': 9, 'batch': 3267, 'loss': 4.206806659698486}
{'epoch': 9, 'batch': 3268, 'loss': 4.639572620391846}
{'epoch': 9, 'batch': 3269, 'loss': 5.019217014312744}
{'epoch': 9, 'batch': 3270, 'loss': 4.901776313781738}
{'epoch': 9, 'batch': 3271, 'loss': 4.6017351150512695}
{'epoch

{'epoch': 9, 'batch': 3403, 'loss': 4.645779609680176}
{'epoch': 9, 'batch': 3404, 'loss': 4.907769203186035}
{'epoch': 9, 'batch': 3405, 'loss': 4.800051212310791}
{'epoch': 9, 'batch': 3406, 'loss': 4.618527889251709}
{'epoch': 9, 'batch': 3407, 'loss': 4.186199188232422}
{'epoch': 9, 'batch': 3408, 'loss': 4.352522373199463}
{'epoch': 9, 'batch': 3409, 'loss': 4.135274887084961}
{'epoch': 9, 'batch': 3410, 'loss': 4.794023513793945}
{'epoch': 9, 'batch': 3411, 'loss': 4.674112796783447}
{'epoch': 9, 'batch': 3412, 'loss': 5.073636054992676}
{'epoch': 9, 'batch': 3413, 'loss': 4.753217697143555}
{'epoch': 9, 'batch': 3414, 'loss': 4.426525592803955}
{'epoch': 9, 'batch': 3415, 'loss': 4.169083118438721}
{'epoch': 9, 'batch': 3416, 'loss': 4.5456624031066895}
{'epoch': 9, 'batch': 3417, 'loss': 4.884456634521484}
{'epoch': 9, 'batch': 3418, 'loss': 5.06331729888916}
{'epoch': 9, 'batch': 3419, 'loss': 4.798466682434082}
{'epoch': 9, 'batch': 3420, 'loss': 4.9023566246032715}
{'epoch':

{'epoch': 9, 'batch': 3553, 'loss': 4.469320297241211}
{'epoch': 9, 'batch': 3554, 'loss': 4.797766208648682}
{'epoch': 9, 'batch': 3555, 'loss': 4.3780293464660645}
{'epoch': 9, 'batch': 3556, 'loss': 4.339352607727051}
{'epoch': 9, 'batch': 3557, 'loss': 3.720871686935425}
{'epoch': 9, 'batch': 3558, 'loss': 2.83424711227417}
{'epoch': 9, 'batch': 3559, 'loss': 4.9008307456970215}
{'epoch': 9, 'batch': 3560, 'loss': 5.215640544891357}
{'epoch': 9, 'batch': 3561, 'loss': 4.457951545715332}
{'epoch': 9, 'batch': 3562, 'loss': 4.2097320556640625}
{'epoch': 9, 'batch': 3563, 'loss': 5.083764553070068}
{'epoch': 9, 'batch': 3564, 'loss': 5.29644775390625}
{'epoch': 9, 'batch': 3565, 'loss': 5.636395454406738}
{'epoch': 9, 'batch': 3566, 'loss': 4.490509986877441}
{'epoch': 9, 'batch': 3567, 'loss': 4.514094829559326}
{'epoch': 9, 'batch': 3568, 'loss': 4.491553783416748}
{'epoch': 9, 'batch': 3569, 'loss': 4.68256950378418}
{'epoch': 9, 'batch': 3570, 'loss': 4.868077278137207}
{'epoch': 

{'epoch': 9, 'batch': 3705, 'loss': 4.899797439575195}
{'epoch': 9, 'batch': 3706, 'loss': 4.55679178237915}
{'epoch': 9, 'batch': 3707, 'loss': 4.375979423522949}
{'epoch': 9, 'batch': 3708, 'loss': 4.498317718505859}
{'epoch': 9, 'batch': 3709, 'loss': 4.524518966674805}
{'epoch': 9, 'batch': 3710, 'loss': 4.654964447021484}
{'epoch': 9, 'batch': 3711, 'loss': 4.098900318145752}
{'epoch': 9, 'batch': 3712, 'loss': 4.046326637268066}
{'epoch': 9, 'batch': 3713, 'loss': 4.981124401092529}
{'epoch': 9, 'batch': 3714, 'loss': 4.925671577453613}
{'epoch': 9, 'batch': 3715, 'loss': 4.610495090484619}
{'epoch': 9, 'batch': 3716, 'loss': 4.616221904754639}
{'epoch': 9, 'batch': 3717, 'loss': 5.075581073760986}
{'epoch': 9, 'batch': 3718, 'loss': 4.707354545593262}
{'epoch': 9, 'batch': 3719, 'loss': 4.793537139892578}
{'epoch': 9, 'batch': 3720, 'loss': 4.992832660675049}
{'epoch': 9, 'batch': 3721, 'loss': 4.592745780944824}
{'epoch': 9, 'batch': 3722, 'loss': 5.207592964172363}
{'epoch': 9

In [28]:
torch.save(model.state_dict(), "model/lstm_pop_positive.pth")

In [7]:
from nltk.tokenize import RegexpTokenizer
text = 'hallo ik \n ben er ook'
text = text.replace('\n', ' Ж ')
print(text)
tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')
tokens = tokenizer.tokenize(text)
tokens

hallo ik  Ж  ben er ook


['hallo', 'ik', 'Ж', 'ben', 'er', 'ook']